In [1]:
# Run this cell to import pyspark and to define start_spark() and stop_spark()

import findspark

findspark.init()

import getpass
import pandas
import pyspark
import random
import re

from IPython.display import display, HTML
from pyspark import SparkContext
from pyspark.sql import SparkSession


# Functions used below

def username():
    """Get username with any domain information removed.
    """

    return re.sub('@.*', '', getpass.getuser())


def dict_to_html(d):
    """Convert a Python dictionary into a two column table for display.
    """

    html = []

    html.append(f'<table width="100%" style="width:100%; font-family: monospace;">')
    for k, v in d.items():
        html.append(f'<tr><td style="text-align:left;">{k}</td><td>{v}</td></tr>')
    html.append(f'</table>')

    return ''.join(html)


def show_as_html(df, n=20):
    """Leverage existing pandas jupyter integration to show a spark dataframe as html.
    
    Args:
        n (int): number of rows to show (default: 20)
    """

    display(df.limit(n).toPandas())

    
def display_spark():
    """Display the status of the active Spark session if one is currently running.
    """
    
    if 'spark' in globals() and 'sc' in globals():

        name = sc.getConf().get("spark.app.name")
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:green">active</span></b>, look for <code>{name}</code> under the running applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'<li><a href="{sc.uiWebUrl}" target="_blank">Spark Application UI</a></li>',
            f'</ul>',
            f'<p><b>Config</b></p>',
            dict_to_html(dict(sc.getConf().getAll())),
            f'<p><b>Notes</b></p>',
            f'<ul>',
            f'<li>The spark session <code>spark</code> and spark context <code>sc</code> global variables have been defined by <code>start_spark()</code>.</li>',
            f'<li>Please run <code>stop_spark()</code> before closing the notebook or restarting the kernel or kill <code>{name}</code> by hand using the link in the Spark UI.</li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))
        
    else:
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:red">stopped</span></b>, confirm that <code>{username() + " (jupyter)"}</code> is under the completed applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))


# Functions to start and stop spark

def start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1):
    """Start a new Spark session and define globals for SparkSession (spark) and SparkContext (sc).
    
    Args:
        executor_instances (int): number of executors (default: 2)
        executor_cores (int): number of cores per executor (default: 1)
        worker_memory (float): worker memory (default: 1)
        master_memory (float): master memory (default: 1)
    """

    global spark
    global sc

    user = username()
    
    cores = executor_instances * executor_cores
    partitions = cores * 4
    port = 4000 + random.randint(1, 999)

    spark = (
        SparkSession.builder
        .master("spark://masternode2:7077")
        .config("spark.driver.extraJavaOptions", f"-Dderby.system.home=/tmp/{user}/spark/")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.executor.instances", str(executor_instances))
        .config("spark.executor.cores", str(executor_cores))
        .config("spark.cores.max", str(cores))
        .config("spark.executor.memory", f"{worker_memory}g")
        .config("spark.driver.memory", f"{master_memory}g")
        .config("spark.driver.maxResultSize", "0")
        .config("spark.sql.shuffle.partitions", str(partitions))
        .config("spark.ui.port", str(port))
        .appName(user + " (jupyter)")
        .getOrCreate()
    )
    sc = SparkContext.getOrCreate()
    
    display_spark()

    
def stop_spark():
    """Stop the active Spark session and delete globals for SparkSession (spark) and SparkContext (sc).
    """

    global spark
    global sc

    if 'spark' in globals() and 'sc' in globals():

        spark.stop()

        del spark
        del sc

    display_spark()


# Make css changes to improve spark output readability

html = [
    '<style>',
    'pre { white-space: pre !important; }',
    'table.dataframe td { white-space: nowrap !important; }',
    'table.dataframe thead th:first-child, table.dataframe tbody th { display: none; }',
    '</style>',
]
display(HTML(''.join(html)))

In [2]:
UESR_DIRECTORY = "hdfs:///user/nki38/outputs/"

In [3]:
LIMITER = False  

In [4]:
#You may increase your resources
#up to 4 executors, 2 cores per executor, 4 GB of executor memory, and 4 GB of master memory.

start_spark(executor_instances=4, executor_cores=2, worker_memory=4, master_memory=4)

spark.dynamicAllocation.enabled,false
spark.executor.instances,4
spark.driver.port,33223
spark.driver.memory,4g
spark.executor.memory,4g
spark.sql.warehouse.dir,file:/users/home/nki38/spark-warehouse/
spark.master,spark://masternode2:7077
spark.executor.id,driver
spark.executor.cores,2
spark.driver.host,mathmadslinux2p.canterbury.ac.nz
spark.sql.shuffle.partitions,32


In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

def spark_read_fixed_width_format(path, spec):
    return (
        spark.read.text(path)
        .select([
            F.trim(F.substring(F.col('value'), start, end - start + 1)).alias(name).cast(type_)
            for name, start, end, type_ in spec
        ])
        .na.replace('', None)
        .repartition(32)
    )




In [6]:
USER_DIRECTORY=f"hdfs:///user/nki38/outputs/ghcnd/"
USER_DIRECTORTY=f"hdfs:///user/nki38/outputs/ghcnd/"
#This is just a typo that's carried through and is proving slightly more difficult to get rid of than you'd think

##### Q1 First it will be helpful to know more about the stations themselves before we study the daily climate summaries in more detail.

In [7]:
path = USER_DIRECTORTY+"stations_augmented.csv"


schema = StructType([
    StructField("ID", StringType(), False),
    StructField("LATITUDE", FloatType(), True),
    StructField("LONGITUDE", FloatType(), True),
    StructField("ELEVATION", FloatType(), True),
    StructField("STATE", StringType(), True),
    StructField("NAME", StringType(), True),
    StructField("GSN_FLAG", StringType(), True),
    StructField("HCN_CRN_FLAG", StringType(), True),
    StructField("WMO_ID", IntegerType(), True),
    StructField("COUNTRY", StringType(), True),
    StructField("min(firstyear)", IntegerType(), True),
    StructField("max(lastyear)", IntegerType(), True),
    StructField("num_elements", IntegerType(), True),
    StructField("num_core_elements", IntegerType(), True),
    StructField("num_precip_elements", IntegerType(), True)
])
 # try with just strings 


    
    
stations_augmented = spark.read.csv(path, schema = schema, header=True)
print("stations augmented containing ", stations_augmented.count(), " Rows")
show_as_html(stations_augmented, 1)


stations augmented containing  125983  Rows


,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN_FLAG,HCN_CRN_FLAG,WMO_ID,COUNTRY,min(firstyear),max(lastyear),num_elements,num_core_elements,num_precip_elements
0,AGM00060555,33.068001,6.089,85.0,None,SIDI MAHDI,None,None,60555,AG,1958,2024,NaN,NaN,NaN


##### How many stations are there in total? How many stations were active so far in 2024?

In [8]:
print(f"total number of stations is {stations_augmented.count()}")

#for the sake of a sanity check
stations_augmented.createOrReplaceTempView("stations")
unique_ids = spark.sql("SELECT COUNT(DISTINCT ID) as UniqueIDs FROM stations")
unique_ids.show()
print(f"The total number of unique IDs is also above, for the sake of sanity checking")


total_2024 = stations_augmented.filter(F.col("max(lastyear)") == 2024)

print("the total number of stations active in 2024 so far are ", total_2024.count())

total number of stations is 125983
+---------+
|UniqueIDs|
+---------+
|   125983|
+---------+

The total number of unique IDs is also above, for the sake of sanity checking
the total number of stations active in 2024 so far are  31837


##### How many stations are in each of the GCOS Surface Network (GSN), the US Historical Climatology Network (HCN), and the US Climate Reference Network (CRN)? Are there any stations that are in more than one of these networks?

In [9]:
rows_with_gsn_flag = stations_augmented.filter(stations_augmented.GSN_FLAG.isNotNull() & (stations_augmented.GSN_FLAG != ""))
rows_with_hcn_flag = stations_augmented.filter(stations_augmented.HCN_CRN_FLAG.isNotNull() & (stations_augmented.HCN_CRN_FLAG == "HCN"))
rows_with_crn_flag = stations_augmented.filter(stations_augmented.HCN_CRN_FLAG.isNotNull() & (stations_augmented.HCN_CRN_FLAG == "CRN"))

show_as_html(rows_with_gsn_flag, 1)
show_as_html(rows_with_hcn_flag, 1)
show_as_html(rows_with_crn_flag, 1)

print("number of stations in GSN", rows_with_gsn_flag.count())
print("number of stations in HCN", rows_with_hcn_flag.count())
print("number of stations in CRN", rows_with_crn_flag.count())

count_both_flags = stations_augmented.agg(F.sum(F.when(F.col("HCN_CRN_FLAG").isNotNull() & F.col("GSN_FLAG").isNotNull(), 1).otherwise(0)).alias("count_both_flags"))

count_both_flags.show()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN_FLAG,HCN_CRN_FLAG,WMO_ID,COUNTRY,min(firstyear),max(lastyear),num_elements,num_core_elements,num_precip_elements
0,AQW00061705,-14.3306,-170.713593,3.7,AS,PAGO PAGO WSO AP,GSN,None,91765,AQ,1945,2024,48,5,1


,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN_FLAG,HCN_CRN_FLAG,WMO_ID,COUNTRY,min(firstyear),max(lastyear),num_elements,num_core_elements,num_precip_elements
0,USC00045032,38.106098,-121.287804,12.2,CA,LODI,None,HCN,NaN,US,1893,2015,23,5,1


,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN_FLAG,HCN_CRN_FLAG,WMO_ID,COUNTRY,min(firstyear),max(lastyear),num_elements,num_core_elements,num_precip_elements
0,USW00003072,32.040798,-100.249397,608.700012,TX,BRONTE 11 NNE,None,CRN,NaN,US,2006,2024,NaN,NaN,NaN


number of stations in GSN 991
number of stations in HCN 1218
number of stations in CRN 234
+----------------+
|count_both_flags|
+----------------+
|              15|
+----------------+



##### Count the total number of stations in each country, and join these counts onto countries so that we can use these counts later if desired.

In [10]:
stations_per_country = stations_augmented.groupBy("COUNTRY").agg(F.count("ID").alias("number_stations"))
print("total number of countries ", stations_per_country.count())
show_as_html(stations_per_country,5)

#Load in countries
countries = spark_read_fixed_width_format(
    path="hdfs:///data/ghcnd/ghcnd-countries.txt",
    spec=[
        ('COUNTRY',      1,  2, StringType()),
        ('COUNTRY_NAME', 4, 64, StringType()),
    ]
)

print ("countries loaded in with ", countries.count(), " rows ")
show_as_html(countries,1)

countries_with_counts = stations_per_country.join(countries, 'COUNTRY', 'left')
show_as_html(countries_with_counts,2)

total number of countries  219


,COUNTRY,number_stations
0,TI,62
1,BB,1
2,CA,9188
3,MX,5249
4,NI,10


countries loaded in with  219  rows 


,COUNTRY,COUNTRY_NAME
0,RW,Rwanda


,COUNTRY,number_stations,COUNTRY_NAME
0,TI,62,Tajikistan
1,BB,1,Barbados


##### Do the same for states and save a copy of each table to your output directory.

In [11]:
stations_per_state = stations_augmented.groupBy("STATE").agg(F.count("ID").alias("number_stations")).filter(F.col("STATE") != "None")
print("total number of states ", stations_per_state.count())
show_as_html(stations_per_state,5)

states = spark_read_fixed_width_format(
    path="hdfs:///data/ghcnd/ghcnd-states.txt",
    spec=[
        ('CODE', 1,  2, StringType()),
        ('NAME', 4, 50, StringType()),
    ]
)
print ("states loaded in with ", states.count(), " rows ")
show_as_html(states,1)

states_with_counts = stations_per_state.join(states, stations_per_state['STATE'] == states['CODE'], 'left')
show_as_html(states_with_counts,2)

countries_with_counts.repartition(1).write.csv(USER_DIRECTORTY + "/countries_with_counts.csv", header=True, mode="overwrite")
states_with_counts.repartition(1).write.csv(USER_DIRECTORTY + "/states_with_counts.csv", header=True, mode="overwrite")

total number of states  75


,STATE,number_stations
0,NT,137
1,CA,3080
2,OK,1081
3,MN,2199
4,NH,471


states loaded in with  74  rows 


,CODE,NAME
0,WI,WISCONSIN


,STATE,number_stations,CODE,NAME
0,NT,137,NT,NORTHWEST TERRITORIES
1,CA,3080,CA,CALIFORNIA


##### How many stations are there in the Southern Hemisphere?

In [12]:
southern_hemisphere_stations = stations_augmented.filter(stations_augmented["LATITUDE"] < 0)
print("the total number of stations in southern hemisphere is ",southern_hemisphere_stations.count() )

the total number of stations in southern hemisphere is  25357


### Q2

### Write a Spark function that computes the geographical distance between two stations using their latitude and longitude as arguments. You can test this function by using CROSS JOIN on a small subset of stations to generate a table with two stations in each row.

In [13]:
import math


def distance(Lat1, Lon1, Lat2, Lon2):
    try:

        if any(val is None for val in [Lat1, Lon1, Lat2, Lon2]):
            return 0

        EarthRadius = 6371  # Earth radius in kilometers
        # to rads for all
        Lat1 = math.radians(Lat1)
        Lat2 = math.radians(Lat2)
        Lon1 = math.radians(Lon1)
        Lon2 = math.radians(Lon2)

        dLon = Lon2 - Lon1
        dLat = Lat2 - Lat1

        a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(Lat1)) * math.cos(math.radians(Lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = EarthRadius * c
        return distance
    except:
        return -1
       # print(f"Error calculating distance: Lat1: {Lat1}, Lon1: {Lon1}, Lat2: {Lat2}, Lon2: {Lon2}")


distance_udf = F.udf(distance, FloatType())


if LIMITER:
    sample = stations_augmented.limit(1)
else:
    sample = stations_augmented.limit(100)




In [14]:

selected_columns = ["ID", "LATITUDE", "LONGITUDE", "NAME"]
only_locs = sample.select(selected_columns)

on_join = only_locs.withColumnRenamed("ID", "new_ID") \
                   .withColumnRenamed("LATITUDE", "new_LATITUDE") \
                   .withColumnRenamed("LONGITUDE", "new_LONGITUDE") \
                   .withColumnRenamed("Name", "new_NAME") \

together = only_locs.join(on_join, how="CROSS", 
                          on=((only_locs.LATITUDE != on_join.new_LATITUDE) & 
                              (only_locs.LONGITUDE != on_join.new_LONGITUDE)))

distances = together.withColumn("Distance_km", distance_udf('LATITUDE','LONGITUDE','new_LATITUDE','new_LONGITUDE'))
show_as_html(distances)

,ID,LATITUDE,LONGITUDE,NAME,new_ID,new_LATITUDE,new_LONGITUDE,new_NAME,Distance_km
0,AGM00060555,33.068001,6.089,SIDI MAHDI,AJ000037756,40.533001,48.932999,MARAZA,4842.605957
1,AGM00060555,33.068001,6.089,SIDI MAHDI,AJ000037923,39.966999,49.400002,ALAT,4882.528809
2,AGM00060555,33.068001,6.089,SIDI MAHDI,AM000037801,40.349998,45.133099,GAVAR,4422.169434
3,AGM00060555,33.068001,6.089,SIDI MAHDI,AMM00037717,40.567001,45.000000,SEVAN OZERO,4412.459961
4,AGM00060555,33.068001,6.089,SIDI MAHDI,AQC00914594,-14.333300,-170.766693,MALAELOA,NaN
5,AGM00060555,33.068001,6.089,SIDI MAHDI,AQW00061705,-14.330600,-170.713593,PAGO PAGO WSO AP,NaN
6,AGM00060555,33.068001,6.089,SIDI MAHDI,AR000087925,-51.617001,-69.282997,RIO GALLEGOS AERO,14555.040039
7,AGM00060555,33.068001,6.089,SIDI MAHDI,ARM00087178,-27.386000,-55.971001,POSADAS,10251.521484
8,AGM00060555,33.068001,6.089,SIDI MAHDI,ASM00094995,-31.542000,159.078995,LORD HOWE ISLAND AERO,NaN
9,AGM00060555,33.068001,6.089,SIDI MAHDI,ASN00001000,-16.291901,127.195602,KARUNJIE,16669.128906


#### Apply this function to compute the pairwise distances between all stations in New Zealand and save the result to your output directory.


###### What two stations are geographically the closest together in New Zealand?

In [15]:
nz_stations = stations_augmented.filter(F.col("ID").substr(1, 2) == 'NZ')

print("there are  ", nz_stations.count() , " nz listings for stations ")
show_as_html(nz_stations, 2)
selected_columns = ["ID", "LATITUDE", "LONGITUDE", "NAME"]


only_locs = nz_stations.select(selected_columns)
on_join = only_locs.withColumnRenamed("ID", "new_ID") \
                   .withColumnRenamed("LATITUDE", "new_LATITUDE") \
                   .withColumnRenamed("LONGITUDE", "new_LONGITUDE") \
                   .withColumnRenamed("Name", "new_NAME") \


together = only_locs.join(on_join, how="Outer", 
                          on=((only_locs.LATITUDE != on_join.new_LATITUDE) & 
                              (only_locs.LONGITUDE != on_join.new_LONGITUDE)))

together = only_locs.join(on_join, how="cross", 
                          on=((only_locs.ID < on_join.new_ID)))

distances = together.withColumn("Distance_km", distance_udf('LATITUDE','LONGITUDE','new_LATITUDE','new_LONGITUDE'))

distances_sorted = distances.orderBy('Distance_km')

show_as_html(distances_sorted)




filename = "nz_stations_distance.csv"
output_path = USER_DIRECTORTY + filename
print(distances_sorted.rdd.getNumPartitions())
distances_sorted.repartition(1).write.csv(output_path, header=True, mode="overwrite")


there are   15  nz listings for stations 


,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN_FLAG,HCN_CRN_FLAG,WMO_ID,COUNTRY,min(firstyear),max(lastyear),num_elements,num_core_elements,num_precip_elements
0,NZ000936150,-42.716999,170.983002,40.0,None,HOKITIKA AERODROME,None,None,93781,NZ,1964,2024,NaN,NaN,NaN
1,NZ000093012,-35.099998,173.266998,54.0,None,KAITAIA,None,None,93119,NZ,1965,2024,NaN,NaN,NaN


,ID,LATITUDE,LONGITUDE,NAME,new_ID,new_LATITUDE,new_LONGITUDE,new_NAME,Distance_km
0,NZ000093417,-40.900002,174.983002,PARAPARAUMU AWS,NZM00093439,-41.333000,174.800003,WELLINGTON AERO AWS,52.270061
1,NZM00093439,-41.333000,174.800003,WELLINGTON AERO AWS,NZM00093678,-42.417000,173.699997,KAIKOURA,171.721252
2,NZM00093678,-42.417000,173.699997,KAIKOURA,NZM00093781,-43.488998,172.531998,CHRISTCHURCH INTL,176.279877
3,NZ000936150,-42.716999,170.983002,HOKITIKA AERODROME,NZM00093781,-43.488998,172.531998,CHRISTCHURCH INTL,192.435684
4,NZ000093417,-40.900002,174.983002,PARAPARAUMU AWS,NZM00093678,-42.417000,173.699997,KAIKOURA,220.920349
5,NZ000093417,-40.900002,174.983002,PARAPARAUMU AWS,NZ000933090,-39.016998,174.182999,NEW PLYMOUTH AWS,227.494232
6,NZ000936150,-42.716999,170.983002,HOKITIKA AERODROME,NZ000937470,-44.516998,169.899994,TARA HILLS,233.585815
7,NZ000933090,-39.016998,174.182999,NEW PLYMOUTH AWS,NZM00093110,-37.000000,174.800003,AUCKLAND AERO AWS,234.539703
8,NZ000933090,-39.016998,174.182999,NEW PLYMOUTH AWS,NZM00093439,-41.333000,174.800003,WELLINGTON AERO AWS,266.510956
9,NZ000093012,-35.099998,173.266998,KAITAIA,NZM00093110,-37.000000,174.800003,AUCKLAND AERO AWS,271.467255


32


##### Q3 Load and count the number of observations in 2023 and then separately in 2024.


In [16]:
schema = StructType([
    StructField("ID", StringType(), False),
    StructField("DATE", IntegerType(), True),
    StructField("ELEMENT", StringType(), True),
    StructField("VALUE", DoubleType(), True),
    StructField("MEASUREMENT FLAG", StringType(), True),
    StructField("QUALITY FLAG", StringType(), True),
    StructField("SOURCE FLAG", StringType(), True),
    StructField("OBSERVATION TIME", StringType(), True) 
])
df_2023 = spark.read.csv("hdfs:///data/ghcnd/daily/2023.csv.gz",schema)
df_2024 = spark.read.csv("hdfs:///data/ghcnd/daily/2024.csv.gz",schema)

if False:
    #just beacuse we dont need these outside of the scope of that question
    print("2023 has  ", df_2023.count())
    print("2024 has ", df_2024.count())

###### Load and count the total number of observations in the years from 2014 to 2023 (inclusive).

In [17]:

if False:
    daily_2014_through_2024 = spark.read.csv("hdfs:///data/ghcnd/daily/{201[4-9]*,202[0-4]*}", schema)
    print("2014 through 2024 has ,daily_2014_through_2024.count()")
    daily_2014_through_2024.show()

##### Count the number of rows in daily

In [18]:
if  LIMITER:
    all_daily = spark.read.csv("hdfs:///data/ghcnd/daily/2023.csv.gz", schema)
    all_daily = all_daily.limit(1000)
    print("limited")
else:
    all_daily = spark.read.csv("hdfs:///data/ghcnd/daily", schema)
    
    print ("unlimited")
    
    

all_daily_len = all_daily.count()
print ("All daily consists of ", all_daily_len, "records")

unlimited
All daily consists of  3103954141 records


##### Filter daily using the filter command to obtain the subset of observations containing the five core elements described in inventory.

In [19]:
core_elements = ['PRCP','SNOW','SNWD','TMAX','TMIN']
filtered_by_elements_daily = all_daily.filter(F.col('ELEMENT').isin(core_elements))
show_as_html(filtered_by_elements_daily,5)
print(filtered_by_elements_daily.count())

,ID,DATE,ELEMENT,VALUE,MEASUREMENT FLAG,QUALITY FLAG,SOURCE FLAG,OBSERVATION TIME
0,AE000041196,20100101,TMAX,259.0,None,None,S,None
1,AE000041196,20100101,TMIN,120.0,None,None,S,None
2,AEM00041194,20100101,TMAX,250.0,None,None,S,None
3,AEM00041194,20100101,TMIN,168.0,None,None,S,None
4,AEM00041194,20100101,PRCP,0.0,None,None,S,None


2631561452


##### How many observations are there for each of the five core elements?


In [20]:
if True:
    num_of_each_element = filtered_by_elements_daily.groupBy('ELEMENT').agg(F.count('*').alias('count'))
    num_of_each_element.show()

+-------+----------+
|ELEMENT|     count|
+-------+----------+
|   SNWD| 297846434|
|   SNOW| 353904309|
|   TMIN| 454759421|
|   PRCP|1069105193|
|   TMAX| 455946095|
+-------+----------+



##### Which element has the most observations?

In [21]:
#see above :)

##### Many stations collect TMIN and TMAX, but do not necessarily report them simultaneously due to issues with data collection or coverage. Determine how many observations of TMIN do not have a corresponding observation of TMAX.

In [22]:
# if False:
    #fine for sample; not fine for daily
#     ONLY_TMAX = all_daily.filter(F.col('ELEMENT') == "TMAX").select('ID','DATE','ELEMENT','VALUE')
#     ONLY_TMIN = all_daily.filter(F.col('ELEMENT') == "TMIN").select('ID','DATE','ELEMENT','VALUE')

#     print(ONLY_TMAX.count())
#     print(ONLY_TMIN.count())

#     joined_df = ONLY_TMIN.join(F.broadcast(ONLY_TMAX), ['ID', 'DATE'], 'left_anti')
#     joined_df.show()


tmin_tmax_df = all_daily.filter((F.col('ELEMENT') == "TMIN") | (F.col('ELEMENT') == "TMAX")) \
                        .select('ID', 'DATE', 'ELEMENT', 'VALUE')
print(tmin_tmax_df.count())

910705516


In [23]:
grouped_df = tmin_tmax_df.groupBy('ID', 'DATE') \
                          .agg(F.collect_set('ELEMENT').alias('ELEMENTS'))
print("total number of contributing rows", grouped_df.count())

total number of contributing rows 465268818


In [24]:
num_tmin_only = grouped_df.filter((F.size('ELEMENTS') == 1) & F.array_contains(grouped_df['ELEMENTS'], 'TMIN'))


print("there are ", num_tmin_only.count(), "rows that are only tmin in", all_daily_len)
#there are  9322723 rows that are only tmin in 3103954141 
#copied here in case the cell is cleared

there are  9322723 rows that are only tmin in 3103954141


In [25]:
#validation
a = all_daily.filter(F.col('ID') == 'AGE00147718')
a.show()

+-----------+--------+-------+-----+----------------+------------+-----------+----------------+
|         ID|    DATE|ELEMENT|VALUE|MEASUREMENT FLAG|QUALITY FLAG|SOURCE FLAG|OBSERVATION TIME|
+-----------+--------+-------+-----+----------------+------------+-----------+----------------+
|AGE00147718|20100101|   TMAX|210.0|            null|        null|          S|            null|
|AGE00147718|20100101|   TMIN| 74.0|            null|        null|          S|            null|
|AGE00147718|20100101|   PRCP|  0.0|            null|        null|          S|            null|
|AGE00147718|20100101|   TAVG|153.0|               H|        null|          S|            null|
|AGE00147718|20100102|   TMAX|196.0|            null|        null|          S|            null|
|AGE00147718|20100102|   PRCP|  0.0|            null|        null|          S|            null|
|AGE00147718|20100102|   TAVG|148.0|               H|        null|          S|            null|
|AGE00147718|20100103|   TMAX|195.0|    

##### How many unique stations contributed to these observations?

In [26]:
if False:
    print(ONLY_TMIN.count())
    print(ONLY_TMAX.count())
    print(joined_df.count())

###### Filter daily to obtain all observations of TMIN and TMAX for all stations in New Zealand, and save the result to your output directory.

In [27]:
show_as_html(all_daily,3)
nz_observations = all_daily.filter((F.col("ID").substr(1, 2) == 'NZ') & (F.col('ELEMENT').isin(['TMIN', 'TMAX'])))
nz_observations.show()
print(nz_observations.count())

,ID,DATE,ELEMENT,VALUE,MEASUREMENT FLAG,QUALITY FLAG,SOURCE FLAG,OBSERVATION TIME
0,AE000041196,20100101,TMAX,259.0,None,None,S,None
1,AE000041196,20100101,TMIN,120.0,None,None,S,None
2,AE000041196,20100101,TAVG,181.0,H,None,S,None


+-----------+--------+-------+-----+----------------+------------+-----------+----------------+
|         ID|    DATE|ELEMENT|VALUE|MEASUREMENT FLAG|QUALITY FLAG|SOURCE FLAG|OBSERVATION TIME|
+-----------+--------+-------+-----+----------------+------------+-----------+----------------+
|NZ000093292|20100101|   TMAX|297.0|            null|        null|          S|            null|
|NZ000093292|20100101|   TMIN| 74.0|            null|        null|          S|            null|
|NZ000093417|20100101|   TMAX|180.0|            null|        null|          S|            null|
|NZ000093417|20100101|   TMIN|125.0|            null|        null|          S|            null|
|NZ000093844|20100101|   TMAX|232.0|            null|        null|          S|            null|
|NZ000093844|20100101|   TMIN| 96.0|            null|        null|          S|            null|
|NZ000933090|20100101|   TMAX|197.0|            null|        null|          S|            null|
|NZ000933090|20100101|   TMIN| 82.0|    

##### How many observations are there, and how many years are covered by the observations?

In [28]:
print('the total number of observations is ', nz_observations.count())

agg_result = nz_observations.agg(F.min("DATE").alias("min_date"), F.max("DATE").alias("max_date"))
agg_result.show()
#Write to disk using implied schema header etc
nz_observations.repartition(1).write.csv(USER_DIRECTORTY + "/NZ_Stations_ANALYSIS.csv", header=True, mode="overwrite")

the total number of observations is  485520
+--------+--------+
|min_date|max_date|
+--------+--------+
|19400308|20240315|
+--------+--------+



In [29]:
!dfs -copyToLocal 

/usr/bin/sh: dfs: command not found


### Group the precipitation observations by year and country. Compute the average rainfall in each year for each country, and save this result to your output directory.

In [30]:
worldwide_precip = all_daily.filter(F.col('ELEMENT') == 'PRCP')

In [31]:
worldwide_precip = worldwide_precip.withColumn("year", F.substring(F.col("DATE").cast("string"), 1, 4))
worldwide_precip = worldwide_precip.withColumn("country", F.substring(F.col("ID").cast("string"), 1, 2))
worldwide_precip.show(1)

+-----------+--------+-------+-----+----------------+------------+-----------+----------------+----+-------+
|         ID|    DATE|ELEMENT|VALUE|MEASUREMENT FLAG|QUALITY FLAG|SOURCE FLAG|OBSERVATION TIME|year|country|
+-----------+--------+-------+-----+----------------+------------+-----------+----------------+----+-------+
|AEM00041194|20100101|   PRCP|  0.0|            null|        null|          S|            null|2010|     AE|
+-----------+--------+-------+-----+----------------+------------+-----------+----------------+----+-------+
only showing top 1 row



In [32]:
avg_measurement_by_year_country = worldwide_precip.groupBy("year", "country") \
    .agg(F.avg("VALUE").alias("avg_measurement"))

In [33]:
print(avg_measurement_by_year_country.count())

17541


In [ ]:





worldwide_precip.show()
avg_measurement_by_year_country.show()
avg_measurement_by_year_country.write.csv(USER_DIRECTORTY + "/rainfall_by_year_country_new.csv", header=True, mode="overwrite")
count = avg_measurement_by_year_country.count()
print("TOTAL IS ", count)

+-----------+--------+-------+-----+----------------+------------+-----------+----------------+----+-------+
|         ID|    DATE|ELEMENT|VALUE|MEASUREMENT FLAG|QUALITY FLAG|SOURCE FLAG|OBSERVATION TIME|year|country|
+-----------+--------+-------+-----+----------------+------------+-----------+----------------+----+-------+
|AEM00041194|20100101|   PRCP|  0.0|            null|        null|          S|            null|2010|     AE|
|AG000060390|20100101|   PRCP|  0.0|            null|        null|          S|            null|2010|     AG|
|AG000060590|20100101|   PRCP|  0.0|            null|        null|          S|            null|2010|     AG|
|AG000060611|20100101|   PRCP|  0.0|            null|        null|          S|            null|2010|     AG|
|AGE00147708|20100101|   PRCP|  5.0|            null|        null|          S|            null|2010|     AG|
|AGE00147716|20100101|   PRCP|  5.0|            null|        null|          S|            null|2010|     AG|
|AGE00147718|201001

In [ ]:
!hdfs dfs -ls /user/nki38/outputs/ghcnd/

In [ ]:
!hdfs dfs -copyToLocal /user/nki38/outputs/ghcnd/

In [ ]:
!wc -l /user/nki38/outputs/ghcnd/NZ_Stations_ANALYSIS.csv


In [ ]:
stop_spark()


